In [7]:
import itertools as it
import numpy as np
import tensorflow as tf
gems = [2, 3, 0, 4, 5]

In [ ]:
# Discards
k = 0
discard_k = list(it.combinations_with_replacement(range(5), k))
discard_k = tf.constant(discard_k, dtype=tf.int8)
discard_k = tf.one_hot(discard_k, depth=5, axis=-1, dtype=tf.int8)
discard_k = tf.reduce_sum(discard_k, axis=1)
discard_k

In [ ]:
# Repeating discards in int32

k = 3
discard_k = list(it.combinations_with_replacement(range(5), k))
discard_k = tf.constant(discard_k, dtype=tf.int32)
discard_k = tf.one_hot(discard_k, depth=5, axis=-1, dtype=tf.int32)
discard_k = tf.reduce_sum(discard_k, axis=1)

# Take 3 different (or less?)
board_gems = [5, 0, 2, 0, 4]

take_3 = list(it.combinations(range(5), 3))
take_3 = tf.constant(take_3, dtype=tf.int32)

board_0_mask = tf.reduce_all(tf.gather(board_gems, take_3) > 0, axis=1)
take_3 = tf.boolean_mask(take_3, board_0_mask)

take_3 = tf.one_hot(take_3, depth=5, axis=-1, dtype=tf.int32)
take_3 = tf.reduce_sum(take_3, axis=1)

combined_3D = take_3[:, None, :] - discard_k[None, :, :]
combined_2D = tf.reshape(combined_3D, (-1, 5))
combined_2D


In [ ]:
# Take 3 different (or less?)
board_gems = [5, 0, 2, 0, 4]

take_3 = list(it.combinations(range(5), 3))
take_3 = tf.constant(take_3, dtype=tf.int8)
take_3 = tf.one_hot(take_3, depth=5, axis=-1, dtype=tf.int8)
take_3 = tf.reduce_sum(take_3, axis=1)

combined_3D = take_3[:, None, :] - discard_k[None, :, :]
combined_2D = tf.reshape(combined_3D, (-1, 5))
combined_2D


In [ ]:
board_gems = [5, 5, 5, 5, 5]
player_gems = [5, 5, 5, 5, 5]

# Discards
k = 3
discard_k = list(it.combinations_with_replacement(range(5), k))
discard_k = tf.constant(discard_k, dtype=tf.int8)
discard_k = tf.one_hot(discard_k, depth=5, axis=-1, dtype=tf.int8)
discard_k = tf.reduce_sum(discard_k, axis=1)


# Take 3 different
take_3 = list(it.combinations(range(5), 3))
take_3 = tf.constant(take_3, dtype=tf.int32)

# Masking out where there are no board gems
respective_board_gems = tf.gather(tf.cast(board_gems, tf.int32), take_3)
board_0_mask = tf.reduce_all(respective_board_gems > 0, axis=1)
take_3 = tf.boolean_mask(take_3, board_0_mask)

take_3 = tf.one_hot(take_3, depth=5, axis=-1, dtype=tf.int8)
take_3 = tf.reduce_sum(take_3, axis=1)


# Take 2 same
take_2_same = tf.eye(5, dtype=tf.int8)*2

# Masking out where there are not at least 4 board gems
board_4_mask = tf.greater_equal(board_gems, 4)
take_2_same = tf.boolean_mask(take_2_same, board_4_mask)


# Combine takes with discards
all_takes = tf.concat([take_3, take_2_same], axis=0)
combined_3D = all_takes[:, None, :] - discard_k[None, :, :]
combined_2D = tf.reshape(combined_3D, (-1, 5)) 
combined_2D

In [ ]:
combined_2D_np = combined_2D.numpy()
unique_rows, counts = np.unique(combined_2D_np, axis=0, return_counts=True)
num_duplicates = np.sum(counts[counts > 1] - 1)

print(f"Total rows: {len(combined_2D_np)}, Duplicate rows: {num_duplicates}")


In [ ]:
board_gems = [5, 5, 5, 5, 5]
player_gems = [5, 5, 5, 5, 5]

# Discards
k = 3
discard_k = list(it.combinations_with_replacement(range(5), k))
discard_k = tf.constant(discard_k, dtype=tf.int8)
discard_k = tf.one_hot(discard_k, depth=5, axis=-1, dtype=tf.int8)
discard_k = tf.reduce_sum(discard_k, axis=1)


# Take 3 different
take_3 = list(it.combinations(range(5), 3))
take_3 = tf.constant(take_3, dtype=tf.int32)

# Masking out where there are no board gems
respective_board_gems = tf.gather(tf.cast(board_gems, tf.int32), take_3)
board_0_mask = tf.reduce_all(respective_board_gems > 0, axis=1)
take_3 = tf.boolean_mask(take_3, board_0_mask)

take_3 = tf.one_hot(take_3, depth=5, axis=-1, dtype=tf.int8)
take_3 = tf.reduce_sum(take_3, axis=1)


# Take 2 same
take_2_same = tf.eye(5, dtype=tf.int8)*2

# Masking out where there are not at least 4 board gems
board_4_mask = tf.greater_equal(board_gems, 4)
take_2_same = tf.boolean_mask(take_2_same, board_4_mask)


# Combine takes with discards
all_takes = tf.concat([take_3, take_2_same], axis=0)
combined_3D = all_takes[:, None, :] - discard_k[None, :, :]
combined_2D = tf.reshape(combined_3D, (-1, 5)) 
combined_2D

In [ ]:
# Theoretical max take lengths
import tensorflow as tf
import itertools as it

# Function to calculate the lengths of all_takes_k
def calculate_all_takes_lengths():
    # Take 3
    take_3 = list(it.combinations(range(5), 3))  # Combinations of 3 distinct gems
    take_3 = tf.constant(take_3, dtype=tf.int32)
    take_3 = tf.one_hot(take_3, depth=5, axis=-1, dtype=tf.int32)
    take_3 = tf.reduce_sum(take_3, axis=1)
    take_3_length = tf.shape(take_3)[0]

    # Take 2: Includes Take 2 same and Take 2 diff
    take_2_diff = list(it.combinations(range(5), 2))  # Combinations of 2 distinct gems
    take_2_diff = tf.constant(take_2_diff, dtype=tf.int32)
    take_2_diff = tf.one_hot(take_2_diff, depth=5, axis=-1, dtype=tf.int32)
    take_2_diff = tf.reduce_sum(take_2_diff, axis=1)
    take_2_diff_length = tf.shape(take_2_diff)[0]

    take_2_same = tf.eye(5, dtype=tf.int8) * 2  # Take 2 same
    take_2_same_length = tf.shape(take_2_same)[0]

    take_2_length = take_2_diff_length + take_2_same_length

    # Take 1
    take_1 = tf.eye(5, dtype=tf.int8)  # Take 1
    take_1_length = tf.shape(take_1)[0]

    print("Theoretical lengths of all_takes_k:")
    print(f"Take 3 (all_takes_3): {take_3_length.numpy()}")
    print(f"Take 2 (all_takes_2): {take_2_length.numpy()}")
    print(f"  - Take 2 diff: {take_2_diff_length.numpy()}")
    print(f"  - Take 2 same: {take_2_same_length.numpy()}")
    print(f"Take 1 (all_takes_1): {take_1_length.numpy()}")

# Call the function to print the results
calculate_all_takes_lengths()


In [ ]:
import numpy as np
import tensorflow as tf
import itertools as it


# Mock simplified version of the Player class
class Player:
    def __init__(self, action_dim=165):
        self.action_dim = action_dim
        self._initialize_all_takes()
        self.gems = np.zeros(6, dtype=int)

    def _initialize_all_takes(self):
        """Preloads all possible take indices."""
        # Take 3
        take_3 = list(it.combinations(range(5), 3))
        take_3 = tf.constant(take_3, dtype=tf.int32)
        take_3 = tf.one_hot(take_3, depth=5, axis=-1, dtype=tf.int32)
        take_3 = tf.reduce_sum(take_3, axis=1)
        self.all_takes_3 = take_3

        # Take 2
        take_2_diff = list(it.combinations(range(5), 2))
        take_2_diff = tf.constant(take_2_diff, dtype=tf.int32)
        take_2_diff = tf.one_hot(take_2_diff, depth=5, axis=-1, dtype=tf.int32)
        take_2_diff = tf.reduce_sum(take_2_diff, axis=1)
        self.all_takes_2_diff = take_2_diff

        self.all_takes_2_same = tf.eye(5, dtype=tf.int32) * 2

        # Take 1
        self.all_takes_1 = tf.eye(5, dtype=tf.int32)

    def _scatter_legal_takes(self, legal_action_mask, board_mask, n_discards, offset):
        """Update the legal action mask."""
        legal_indices = tf.where(board_mask)
        legal_indices = tf.cast(tf.reshape(legal_indices, [-1]), tf.int32)
        if tf.size(legal_indices) == 0:
            print("No valid moves. Skipping.")
            return legal_action_mask

        action_indices = offset + legal_indices * n_discards + n_discards
        action_indices = tf.reshape(action_indices, [-1, 1])
        action_updates = tf.ones(tf.shape(action_indices)[0], dtype=tf.bool)

        # Debug prints
        print("legal_indices:", legal_indices.numpy())
        print("action_indices:", action_indices.numpy())
        print("action_updates:", action_updates.numpy())

        legal_action_mask = tf.tensor_scatter_nd_update(legal_action_mask, action_indices, action_updates)
        return legal_action_mask

    def _get_legal_takes(self, board_gems):
        """Get the legal takes."""
        n_gems = self.gems.sum()
        board_gems = tf.constant(board_gems[:5], dtype=tf.int32)
        legal_take_mask = tf.zeros([self.action_dim], dtype=tf.bool)
        offset = 0

        # Take 3
        n_discards = max(0, -7 + n_gems)
        board_gt0_mask = board_gems > 0
        takes_mask = tf.cast(self.all_takes_3, tf.bool)
        takes_ltboard_mask = tf.reduce_all(takes_mask & board_gt0_mask, axis=1)
        legal_take_mask = self._scatter_legal_takes(legal_take_mask, takes_ltboard_mask, n_discards, offset)
        offset += tf.shape(takes_ltboard_mask)[0]

        # Return final legal_take_mask
        return legal_take_mask


# Standalone test
player = Player()

# Input gems and board setup
player_gems = [1, 1, 1, 0, 0, 1]  # Player's gems (last is gold)
board_gems = [2, 0, 3, 5, 0]      # Gems available on the board
player.gems = np.array(player_gems)

# Test the _get_legal_takes function
print("Testing _get_legal_takes")
legal_take_mask = player._get_legal_takes(board_gems)
print("\nResulting legal_take_mask:")
print(legal_take_mask.numpy())
